In [5]:
"""
A small library for specifying neural networks using index notation similar to Einstein summation notation.
"""

import numpy as np
import jax.numpy as jnp
import uuid
from IPython.display import display, Math

/Users/isaacbreen/miniforge3/lib/python3.9/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [6]:
class IndexExpression:
    """
    A class for representing index expressions
    """
    def __init__(self, operation, subexpressions):
        self.operation = operation
        self.subexpressions = subexpressions
        
    def __eq__(self, other):
        return self.operation == other.operation and self.subexpressions == other.subexpressions
    
    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return str(self.operation) + '(' + ','.join(str(s) for s in self.subexpressions) + ')'

    def __repr__(self):
        return str(self)
    
    def __add__(self, other):
        return IndexExpression('+', [self, other])
    
    def __radd__(self, other):
        return IndexExpression('+', [other, self])
    
    def __sub__(self, other):
        return IndexExpression('-', [self, other])
    
    def __rsub__(self, other):
        return IndexExpression('-', [other, self])
    
    def __mul__(self, other):
        return IndexExpression('*', [self, other])
    
    def __rmul__(self, other):
        return IndexExpression('*', [other, self])
    
    def __truediv__(self, other):
        return IndexExpression('/', [self, other])
    
    def __rtruediv__(self, other):
        return IndexExpression('/', [other, self])
    
    def __pow__(self, other):
        return IndexExpression('**', [self, other])
    
    def __rpow__(self, other):
        return IndexExpression('**', [other, self])

    def __neg__(self):
        return IndexExpression('-', [self])
    
    def __pos__(self):
        return IndexExpression('+', [self])
    
    def __abs__(self):
        return IndexExpression('abs', [self])
    
    def __invert__(self):
        return IndexExpression('~', [self])
    
    def __call__(self, *args):
        return IndexExpression('()', [self] + list(args))
        
    def __lt__(self, other):
        return IndexExpression('<', [self, other])
            
    def __le__(self, other):
        
        return IndexExpression('<=', [self, other])
    
    def __gt__(self, other):
        return IndexExpression('>', [self, other])
    
    def __ge__(self, other):
        return IndexExpression('>=', [self, other])
    
    def __eq__(self, other):
        return IndexExpression('==', [self, other])
    
    def __ne__(self, other):
        return IndexExpression('!=', [self, other])
    
    def __and__(self, other):
        return IndexExpression('&', [self, other])
    
    def __or__(self, other):
        return IndexExpression('|', [self, other])
    
    def __xor__(self, other):
        return IndexExpression('^', [self, other])
        
    def __getstate__(self):
        return self.operation, self.subexpressions
    
    def __setstate__(self, state):
        self.operation, self.subexpressions = state
        
    def to_latex(self, prev_precedence=0):
        """
        Convert the expression to LaTeX
        """
        precedence = {
            '==': 1,
            '!=': 1,
            '<': 2,
            '<=': 2,
            '>': 2,
            '>=': 2,
            '+': 3,
            '-': 3,
            '*': 4,
            '/': 0,
            '**': 5,
            '&': 6,
            '|': 6,
            '^': 0,
            '~': 6,
            '()': 7,
            'abs': -1,
        }
        operators = {
            '+': ['', '+', ''],
            '-': ['', '-', ''],
            '*': ['', ' \\cdot ', ''],
            '/': ['\\frac{', '}{', '}'],
            '**': ['', '^{', '}'],
            '<': ['', '<', ''],
            '<=': ['', ' \\leq ', ''],
            '>': ['', '>', ''],
            '>=': ['', ' \\geq ', ''],
            '==': ['', '=', ''],
            '!=': ['', ' \\neq ', ''],
            '&': ['', ' \\wedge ', ''],
            '|': ['', ' \\vee ', ''],
            '^': ['', ' \\oplus ', ''],
            '()': ['(', ')', ''],
            '[]': ['[', ']', ''],
            'abs': ['\\left|', '', '\\right|'],
            '~': ['\\left\\lnot', '', ''],
            '<>': ['', ' \\neq ', ''],
            '<': ['', '<', ''],
            '>': ['', '>', ''],
            '<=': ['', ' \\leq ', ''],
            '>=': ['', ' \\geq ', ''],
            '==': ['', '=', ''],
            '!=': ['', ' \\neq ', ''],
        }
        
        if isinstance(self, IndexExpression):
            s = ''
            for i, subexpression in enumerate(self.subexpressions):
                if i > 0:
                    s += operators[self.operation][1]
                if isinstance(subexpression, IndexExpression):
                    s += subexpression.to_latex(precedence[self.operation])
                else:
                    s += str(subexpression)
            s = operators[self.operation][0] + s + operators[self.operation][2]
            # If this operator (e.g. *) binds more tightly than its parent (e.g. +), parenthesis are not needed. Otherwise, they are.
            if precedence[self.operation] == -1:
                pass
            elif precedence[self.operation] < prev_precedence:
                s = '\\left(' + s + '\\right)'
            return s
        else:
            return str(self)
        
        
                
                
class Indices(IndexExpression):
    """
    A class for representing expressions involving indices
    """
    def __init__(self, indices):
        self.indices = indices
        self.realised = False

    def __str__(self):
        return ','.join(str(i) for i in self.indices)

    def __repr__(self):
        return str(self)
    
    def to_latex(self, prev_precedence=0):
        s = self.indices[0] + r"_{"
        for i, index in enumerate(self.indices[1:]):
            if isinstance(index, tuple):
                s += str(index[0]) + r"_{" + str(index[1]) + r"}"
            else:
                s += str(index)
            if i < len(self.indices) - 2:
                s += r" "
        s += r"}"
        return s
    
    def __eq__(self, other):
        return self.indices == other.indices

    def __hash__(self):
        return hash(str(self))
    

class Size(IndexExpression):
    """
    A class for representing expressions involving sizes
    """
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return '||' + self.name + '||'

    def __repr__(self):
        return str(self)
    
    def to_latex(self, prev_precedence=0):
        return r"\|" + self.name + r"\|"

    def __hash__(self):
        return hash(str(self))


class Realised(IndexExpression):
        """
        Represents realised values of an index. For example, $x(t) = x_0 e^{-t}$ would be represented as `ie['x','t'] = ie['x_0',{'name':'t', 'value':0}] * np.euler**(-ie['t']).value()`
        """
        def __init__(self, name):
            self.name = name
            self.realised = True

        def __str__(self):
            return str(self.name)

        def __repr__(self):
            return str(self)
        
        def to_latex(self, prev_precedence=0):
            return str(self.name)

        def __hash__(self):
            return hash(str(self))


class Concatenate(IndexExpression):
    """
    A class for representing concatenation
    """
    def __init__(self, from_indices, to_indices):
        self.from_indices = from_indices
        self.to_indices = to_indices
        self.realised = False

    def __str__(self):
        return ','.join(str(i) for i in self.from_indices) + '->' + ','.join(str(i) for i in self.to_indices)

    def __repr__(self):
        return str(self)
    
    def to_latex(self, prev_precedence=0):
        # Concatenation operator: $\mathbin{+\mkern-10mu+}_{abc \to d}
        return r"\mathbin{+\mkern-10mu+}_{{" + ' '.join(i[0] + r"_{" + str(i[1]) + r"}" if isinstance(i, tuple) else str(i) for i in self.from_indices.indices) + r"} \to {" + ' '.join(i[0] + r"_{" + str(i[1]) + r"}" if isinstance(i, tuple) else str(i) for i in self.to_indices.indices) + r"}}"
    
    def __eq__(self, other):
        return self.from_ == other.from_ and self.to_ == other.to_

    def __hash__(self):
        return hash(str(self))
    
    
class Function(IndexExpression):
    """
    Represents a function
    """
    def __init__(self, name, function=None):
        self.name = name
        self.function = function
        self.parameters = None
                
    def __str__(self):
        return self.name + '(' + ','.join(str(i) for i in self.parameters) + ')'
    
    def __repr__(self):
        return str(self)

    def to_latex(self, prev_precedence=0):
        return r"\mathrm{" + self.name + r"}" + r"\left(" + ','.join(p.to_latex() for p in self.parameters) + r"\right)"
    
    def __eq__(self, other):
        return self.name == other.name and self.parameters == other.parameters
    
    def __hash__(self):
        return hash(str(self))
    
    def __call__(self, *args):
        self.parameters = args
        return self
        

class IndexEquations:
    """
    Represents equations in index notation. In essence, it assigns each sequence of indices to an expression of indices.
    """
    def __init__(self):
        self.equations = {}

    def __setitem__(self, names, value):
        """
        Define a variable as a sequence of indices.
        """
        if isinstance(names, str):
            names = tuple([s.strip() for s in names.split(",") if s.strip() != ""])
        if isinstance(names, tuple):
            indices = Indices(names)
        if isinstance(names, Indices):
            indices = names
        self.equations[indices] = value

    def __getitem__(self, names):
        """
        Get an index representation of a variable as a sequence of indices.
        """
        if isinstance(names, str):
            names = tuple([s.strip() for s in names.split(",") if s.strip() != ""])
        return Indices(names)
    
    def get_realisation(self, name):
        """
        Get a representation of a realisation of a variable.
        """
        return Indices([name]).value()
    
    def get_size(self, name):
        """
        Get the size of a variable.
        """
        return Size(name)

    def __str__(self):
        s = "\\begin{align*}\n"
        for k, v in self.equations.items():
            s += k.to_latex() + " &= " + v.to_latex() + " \\\\\n"
        s += "\\end{align*}"
        return s
    
"""
$$
\begin{align*}
q_{b h t c} &= t \\
k_{b h t c} &= t \\
v_{b h t c} &= x_{b t c} \\
a_{b h t_{0} t_{1}} &= \left|k_{t_{0} c}-k_{t_{1} c}\right| \leq 1 \\
u^2_{b t_{0} c} &= U_{c_{0} c_{1}} \cdot a_{h t_{0} t_{1}} \cdot v_{h t_{1} c_{0}} \\
u^1_{b t c} &= x_{b t c}+u^1_{b t c} \\
u^0_{b t c} &= \left(\frac{\gamma_{c} \cdot \left(u^1_{b t c}-\mu_{c}\right)}{\sigma_{c}}\right)+\beta_{c} \\
\mu_{c} &= \left(\frac{1_{b t}}{\|b\| \cdot \|t\|}\right) \cdot u^1_{b t c} \\
\sigma_{c} &= \left(\left(\frac{1_{b t}}{\|b\| \cdot \|t\|}\right) \cdot \left(u^1_{b t c}-\mu_{c}\right)^{2}\right)^{0.5} \\
z_{b c} &= \left(\frac{\gamma_{c} \cdot \left(z_{b c}-\mu_{c}\right)}{\sigma_{c}}\right)+\beta_{c} \\
\end{align*}
$$
"""

k = 1
ie = IndexEquations()
v = Realised
s = Size
ie['q,b,h,t,c'] = v('t')
ie['k,b,h,t,c'] = v('t')
ie['v,b,h,t,c'] = ie['x,b,t,c']
ie['a','b','h',('t',0), ('t',1)] = abs(ie['k',('t',0),'c'] - ie['k',('t',1),'c']) <= k
ie["u^2",'b',('t',0),'c'] = ie['U',('c',0),('c',1)] * ie['a','h',('t',0),('t',1)] * ie['v','h',('t',1),('c',0)]
ie["u^1",'b','t','c'] = ie['x','b','t','c'] + ie["u^1",'b','t','c']
ie["u^0",'b','t','c'] = ie['\\gamma','c'] * (ie['u^1','b','t','c'] - ie['\\mu','c']) / ie['\\sigma','c'] + ie['\\beta','c']
ie['\\mu','c'] = ie['1','b','t'] / (s('b') * s('t')) * ie["u^1",'b','t','c']
ie['\\sigma','c'] = (ie['1','b','t'] / (s('b') * s('t')) * (ie["u^1",'b','t','c'] - ie['\\mu','c']) ** 2) ** 0.5
ie['z','b','c'] = ie['\\gamma','c'] * (ie['z','b','c'] - ie['\\mu','c']) / ie['\\sigma','c'] + ie['\\beta','c']

display(Math(str(ie)))
# print(str(ie))

<IPython.core.display.Math object>

In [289]:
# Generative Pretrained Transformer
s = \
r"""
\newcommand\mdoubleplus{\mathbin{+\mkern-10mu+}}
$$
\begin{align*}
q_{b h t s} &= Q_{h s c} x_{b t c} \\
k_{b h t s} &= K_{h s c} x_{b t c} \\
v_{b h t s} &= V_{h s c} x_{b t c} \\
\alpha_{b h t_0 t_1} &= q_{b h t_0 s} \cdot k_{b h t_1 s} \\
u^2_{b t_0 c_0} &= U_{c_0 c_1} \cdot \mdoubleplus_{hs \to c_1} \cdot \alpha_{b h t_0 t_1} \cdot v_{b h t_1 s} \\
u^1_{b t c} &= x_{b t c} + u^2_{b t c} \\
u^0_{b t c} &= \frac{\gamma^u_{c} \cdot \left(u^1_{b t c}-\mu^u_{c}\right)}{\sigma^u_{c}}+\beta^u_{c} \\
\mu^u_{c} &= \frac{ 1_{b t} u^1_{b t c} }{\|b\| \cdot \|t\|} \\
\sigma^u_{c} &= \sqrt{ \frac{1_{b t} \left(u^1_{b t c}-\mu^u_{c}\right)^{2} }{\|b\| \cdot \|t\|}} \\
z^2_{b t c^2} &= \mathrm{GELU} \left( Z^2_{c^2 c} \cdot u^0_{b t c} \right) \\
z^1_{b t c} &= Z^1_{c c^2} \cdot z^2_{b t c^2} \\
z^0_{b t c} &= \frac{\gamma^z_{c} \cdot \left(z^1_{b t c}-\mu^z_{c}\right)}{\sigma^z_{c}}+\beta^z_{c} \\
\mu^z_{c} &= \frac{ 1_{b t} z^1_{b t c} }{\|b\| \cdot \|t\|} \\
\sigma^z_{c} &= \sqrt{ \frac{1_{b t} \left(z^1_{b t c}-\mu^z_{c}\right)^{2} }{\|b\| \cdot \|t\|}} \\
\end{align*}
$$
"""
display(Math(s))

ie = IndexEquations()
v = Realised
s = Size
ie['q,b,h,t,s'] = ie['Q','h','s','c'] * ie['x','b','t','c']
ie['k,b,h,t,s'] = ie['K','h','s','c'] * ie['x','b','t','c']
ie['v,b,h,t,s'] = ie['V','h','s','c'] * ie['x','b','t','c']
ie['\\alpha','b','h',('t',0),('t',1)] = ie['q','b','h',('t',0),'s'] * ie['k','b','h',('t',1),'s']
ie['u^2','b','t','c'] = ie['U','c',('c',1)] * Concatenate(ie['h','s'], ie[('c',1),]) * ie['\\alpha','b','h',('t',0),('t',1)] * ie['v','b','h',('t',1),'s']
ie['u^1','b','t','c'] = ie['x','b','t','c'] + ie['u^2','b','t','c']
ie['u^0','b','t','c'] = ie['\\gamma','c'] * (ie['u^1','b','t','c'] - ie['\\mu','c']) / ie['\\sigma','c'] + ie['\\beta','c']
ie['\\mu^u','c'] = ( ie['1','b','t'] * ie['u^1','b','t','c'] ) / (s('b') * s('t'))
ie['\\sigma^u','c'] = ( ie['1','b','t'] * (ie['u^1','b','t','c'] - ie['\\mu','c']) ** 2 ) ** 0.5
ie['z^2','b','t','c^2'] = Function("GELU")(ie['Z^2','c^2','c'] * ie['u^0','b','t','c'])
ie['z^1','b','t','c'] = ie['Z^1','c','c^2'] * ie['z^2','b','t','c^2']
ie['z^0','b','t','c'] = ie['\\gamma','c'] * (ie['z^1','b','t','c'] - ie['\\mu^z','c']) / ie['\\sigma^z','c'] + ie['\\beta','c']
ie['\\mu^z','c'] = ( ie['1','b','t'] * ie['z^1','b','t','c'] ) / (s('b') * s('t'))
ie['\\sigma^z','c'] = ( ( ie['1','b','t'] * (ie['z^1','b','t','c'] - ie['\\mu^z','c']) ** 2 ) / (s('b') * s('t')) ) ** 0.5

display(Math(str(ie)))
print(str(ie))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

\begin{align*}
q_{b h t s} &= Q_{h s c} \cdot x_{b t c} \\
k_{b h t s} &= K_{h s c} \cdot x_{b t c} \\
v_{b h t s} &= V_{h s c} \cdot x_{b t c} \\
\alpha_{b h t_{0} t_{1}} &= q_{b h t_{0} s} \cdot k_{b h t_{1} s} \\
u^2_{b t c} &= U_{c c_{1}} \cdot \mathbin{+\mkern-10mu+}_{{h s} \to {c_{1}}} \cdot \alpha_{b h t_{0} t_{1}} \cdot v_{b h t_{1} s} \\
u^1_{b t c} &= x_{b t c}+u^2_{b t c} \\
u^0_{b t c} &= \left(\frac{\gamma_{c} \cdot \left(u^1_{b t c}-\mu_{c}\right)}{\sigma_{c}}\right)+\beta_{c} \\
\mu^u_{c} &= \frac{1_{b t} \cdot u^1_{b t c}}{\|b\| \cdot \|t\|} \\
\sigma^u_{c} &= \left(1_{b t} \cdot \left(u^1_{b t c}-\mu_{c}\right)^{2}\right)^{0.5} \\
z^2_{b t c^2} &= \mathrm{GELU}\left(Z^2_{c^2 c} \cdot u^0_{b t c}\right) \\
z^1_{b t c} &= Z^1_{c c^2} \cdot z^2_{b t c^2} \\
z^0_{b t c} &= \left(\frac{\gamma_{c} \cdot \left(z^1_{b t c}-\mu^z_{c}\right)}{\sigma^z_{c}}\right)+\beta_{c} \\
\mu^z_{c} &= \frac{1_{b t} \cdot z^1_{b t c}}{\|b\| \cdot \|t\|} \\
\sigma^z_{c} &= \left(\frac{1_{b t} 

In [292]:
# Transformer
s = \
r"""
\newcommand\mdoubleplus{\mathbin{+\mkern-10mu+}}
$$
\begin{align*}
q_{b h t c^e} &= Q_{h c^e c^x} x_{b t c^x} \\
k_{b h t c^e} &= K_{h c^e c^x} x_{b t c^x} \\
v_{b h t c^v} &= V_{h s c^v} x_{b t c^x} \\
\alpha_{b h t_0 t_1} &= q_{b h t_0 c^e} \cdot k_{b h t_1 c^e} \\
u^2_{b t_0 c^x} &= U_{c^x h c^v} \cdot \alpha_{b h t_0 t_1} \cdot v_{b h t_1 c^v} \\
u^1_{b t c^x} &= x_{b t c^x} + u^2_{b t c^x} \\
u^0_{b t c^x} &= \frac{\gamma^u_{c^x} \cdot \left(u^1_{b t c^x}-\mu^u_{c^x}\right)}{\sigma^u_{c^x}}+\beta^u_{c^x} \\
\mu^u_{c} &= \frac{ 1_{b t} u^1_{b t c} }{\|b\| \cdot \|t\|} \\
\sigma^u_{c} &= \sqrt{ \frac{1_{b t} \left(u^1_{b t c}-\mu^u_{c}\right)^{2} }{\|b\| \cdot \|t\|}} \\
z^2_{b t c^2} &= \mathrm{GELU} \left( Z^2_{c^2 c} \cdot u^0_{b t c} \right) \\
z^1_{b t c} &= Z^1_{c c^2} \cdot z^2_{b t c^2} \\
z^0_{b t c} &= \frac{\gamma^z_{c} \cdot \left(z^1_{b t c}-\mu^z_{c}\right)}{\sigma^z_{c}}+\beta^z_{c} \\
\mu^z_{c} &= \frac{ 1_{b t} z^1_{b t c} }{\|b\| \cdot \|t\|} \\
\sigma^z_{c} &= \sqrt{ \frac{1_{b t} \left(z^1_{b t c}-\mu^z_{c}\right)^{2} }{\|b\| \cdot \|t\|}} \\
\end{align*}
$$
"""
display(Math(s))

ie = IndexEquations()
v = Realised
s = Size


<IPython.core.display.Math object>

In [277]:
# Convolutional Neural Network with kernel radius $k$
s = \
"""
ie['q,b,h,t,c^e'] = v('t')
ie['k,b,h,t,c^e'] = v('t')
ie['v,b,h,t,c'] = ie['x,b,t,c']
ie['\\alpha,b,h,(t,0),(t,1)'] = \| ie['q','b','h',('t',0),'c^e'] - ie['k','b','h',('t',1),'c^e'] \|_1 <= k
ie['u^2,b,t,c'] = ie['U','c',('c',1)] * Concatenate(ie['h','s'], ie[,]) * ie['\\alpha,b,h,t,(t,1)'] * ie['v,b,h,(t,1),c']

"""

AttributeError: 'X' object has no attribute 'abc'